In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
pip install transformers

     |████████████████████████████████| 2.1MB 18.9MB/s 
     |████████████████████████████████| 901kB 52.0MB/s 
     |████████████████████████████████| 3.3MB 54.5MB/s 


In [4]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures
import pandas as pd
import tensorflow as tf

model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Summary of the Model

In [5]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  1538      
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


## Loading Data (Training & Testing)

In [6]:
test = pd.read_csv('/content/drive/MyDrive/TFG/BERT/tagged_data_finalin/man_test_data.csv')

In [7]:
# Change type of label column from double to int
# test['clean_post'] = test['clean_post'].str.decode("utf-8")
test['label'] = test['label'].replace([-1], 0)
test['label'] = test['label'].astype(int)
test.head()

,clean_post,label
0,Agreed Ross. Take for example $TSLA at 300 was...,1
1,I really regret buying a Tesla. Said no Tesla ...,1
2,UP 100% AND IM STILL NOT FUCKING LEAVING! 💎🤲💎🤲...,1
3,"After today's rally, S&amp;P 500 needs a gain ...",1
4,"For the S&amp;P 500, today marks the peak day ...",1


In [8]:
from collections import Counter
print(Counter(test['label'].tolist()))

Counter({1: 100, 0: 100})


In [9]:
train = pd.read_csv('/content/drive/MyDrive/TFG/data/tagged_data/man_training_data.csv')

In [10]:
# train['clean_post'] = train['clean_post'].str.decode("utf-8")
train['label'] = train['label'].replace([-1], 0)
train.head()

,clean_post,label
0,The world's most innovative companies are obse...,1
1,Just a bunch of degenerates dunking on $TSLA,0
2,$TSLA - absolutely. This fraud is not only ge...,0
3,If Elon is headed to #Shanghai for groundbreak...,1
4,BREAKING: Tesla researchers publish work on hy...,1


In [11]:
# # Remove and rename columns
# train= train[['clean_tweet', 'label']]
# train = train.rename(columns={'clean_tweet':'clean_post'})
# train.head()
print(Counter(train['label'].tolist()))

Counter({1: 400, 0: 400})


In [12]:
print(train.isna().sum())
# Remove na 
train = train.dropna()
print(train.isna().sum())

clean_post    0
label         0
dtype: int64
clean_post    0
label         0
dtype: int64


## Creating Input Sequences

In [13]:
def convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN): 
  train_InputExamples = train.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis=1)

  validation_InputExamples = test.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis=1)
  
  return train_InputExamples, validation_InputExamples

  train_InputExamples, validation_InputExamples = convert_data_to_examples(train, 
                                                                           test, 
                                                                           'DATA_COLUMN', 
                                                                           'LABEL_COLUMN')
  
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = [] # -> will hold InputFeatures to be converted later

    for e in examples:
        # Documentation is really strong for this method, so please take a look at it
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length, # truncates if len(s) > max_length
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True, # pads to the right by default # CHECK THIS for pad_to_max_length
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
            input_dict["token_type_ids"], input_dict['attention_mask'])

        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )


DATA_COLUMN = 'clean_post'
LABEL_COLUMN = 'label'

In [14]:
train_InputExamples, validation_InputExamples = convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN)

train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.shuffle(100).batch(32).repeat(2)

validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(32)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


## Fine-Tuning BERT Model

In [15]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

model.fit(train_data, epochs=4, validation_data=validation_data)

Epoch 1/4
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autogr

In [16]:
# model.save('/content/drive/MyDrive/TFG/BERT/10_min/10_tf_final_model.model')

In [17]:
# model.save_pretrained("/content/drive/MyDrive/TFG/BERT/10_min/10_final_model.model")

In [22]:
val_df = pd.read_csv('/content/drive/MyDrive/TFG/BERT/tagged_data_finalin/man_test_data.csv')
val_df['label'] = val_df['label'].replace([-1], 0)
val_df.tail(20)

,clean_post,label
180,As analysts rush to slash $TSLA 2020 GAAP net ...,0
181,Traders are shorting the S&amp;P 500 at a rate...,0
182,"Self damaging cars, is there nothing $Tsla can...",0
183,CCIV shorted,0
184,🇺🇸 S&amp;P 500 vs. 12-Month Forward EPS Estima...,0
185,"And even the lawsuit about the hidden, potenti...",0
186,Ratio of Transports to the S&amp;P 500 at its ...,0
187,"Dow drops more than 400 points, S&amp;P 500 po...",0
188,$TSLA return since Battery Day: 0% 🤯,0
189,Agree. Per my pinned tweet I think the real ca...,0


In [19]:
pred_sentences = val_df['clean_post'].tolist()
true_label = val_df['label'].tolist()
print(true_label)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [20]:
tf_batch = tokenizer(pred_sentences, max_length=128, padding=True, truncation=True, return_tensors='tf')
tf_outputs = model(tf_batch)
tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
labels = [0,1]
label = tf.argmax(tf_predictions, axis=1)

In [26]:
pred_label = label.numpy().tolist()
print(pred_label[80:])

[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [31]:
wrong_i = []
for i in range(len(true_label)):
  if true_label[i] != pred_label[i]:
    wrong_i.append(i)
print(wrong_i)

[1, 6, 7, 11, 14, 18, 32, 33, 52, 65, 71, 78, 83, 104, 113, 114, 116, 118, 122, 127, 134, 157, 161, 188, 190]


In [33]:
for i in wrong_i:
  print(pred_sentences[i])
  print()

I really regret buying a Tesla. Said no Tesla owner ever.  The only regrets I and other owners have is not buying the higher spec version  And for those with the highest spec, not having the other models too.  Who doesnt want S3XY Roadster and Pickup on their driveway? $TSLA

The S&amp;P 500 is on the verge of finishing off its strongest quarter in a decade. This comes after one of the worst quarters in the past decade making for a stark contrast in behavior over the past 6 months.Get your analysis here from  here:

When is $TSLA going to replace the Nasdaq as the benchmark?

S&amp;P 500 Futures. The only way is up? 2865 and 2900 last two big resistances IMO

Meanwhile lockdowns and presidential confusion.  What better time to add to the S&amp;P 500?  When you LEAST expect it.  That’s why you buy and wait.  $tsla.

I can’t watch the market all day to avoid holding the bag later. What’s a good exit point that allow me to get back to my day job?

The S&amp;P 500 is +11.40% YTD while Berk

In [34]:
from sklearn import metrics

In [35]:
f_score = metrics.f1_score(true_label, pred_label)
print(f'F Score: {f_score}')

F Score: 0.8743718592964824


In [36]:
accuracy = metrics.accuracy_score(true_label, pred_label)
print(f'Accuracy: {accuracy}')

Accuracy: 0.875


In [37]:
metrics.confusion_matrix(true_label, pred_label)

array([[88, 12],
       [13, 87]])

In [42]:

pred_sentences = ['Im bearish',
                  ' TSLA is down']

In [43]:
tf_batch = tokenizer(pred_sentences, max_length=128, padding=True, truncation=True, return_tensors='tf')
tf_outputs = model(tf_batch)
print(tf_outputs)
tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
print(tf_predictions)
labels = [0,1]
label = tf.argmax(tf_predictions, axis=1)

print()
print(label)
print(label.numpy())
label = label.numpy()
for i in range(len(pred_sentences)):
  print(pred_sentences[i], ": \n", labels[label[i]])


TFSequenceClassifierOutput(loss=None, logits=<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[ 3.1626718, -3.1631267],
       [ 3.46866  , -3.4545083]], dtype=float32)>, hidden_states=None, attentions=None)
tf.Tensor(
[[9.9821365e-01 1.7863399e-03]
 [9.9901628e-01 9.8373659e-04]], shape=(2, 2), dtype=float32)

tf.Tensor([0 0], shape=(2,), dtype=int64)
[0 0]
Im bearish : 
 0
 TSLA is down : 
 0


In [44]:
TSLA_tweets = pd.read_csv('/content/drive/MyDrive/TFG/BERT/tweets_posts/clean_2020_SPY_tweets.csv', lineterminator='\n')

In [45]:
TSLA_tweets.head()

,date,tweet,clean_post
0,2020-01-02,$SPY #SPY Daily vs. Monthly https://t.co/uOqC...,$SPY #SPY Daily vs. Monthly
1,2020-01-02,What a relaxing day off. Best part is the $SPY...,What a relaxing day off. Best part is the $SPY...
2,2020-01-02,$SPX managed to eek out a close above that LT ...,$SPX managed to eek out a close above that LT ...
3,2020-01-02,Mercury Bands &amp; Algorithmic Pivots ES Futu...,Mercury Bands &amp; Algorithmic Pivots ES Futu...
4,2020-01-02,2020's Key Global Events: A Calendar For The Y...,2020's Key Global Events: A Calendar For The Y...


In [46]:
print(TSLA_tweets.isna().sum())
# Remove na 
TSLA_tweets = TSLA_tweets.dropna()
print(TSLA_tweets.isna().sum())

date          0
tweet         0
clean_post    0
dtype: int64
date          0
tweet         0
clean_post    0
dtype: int64


In [47]:
print(len(TSLA_tweets.index))

131870


In [48]:
tsla_sentences = TSLA_tweets['clean_post'].tolist()

In [49]:
print(type(tsla_sentences[88]))

<class 'str'>


In [50]:
from tqdm import tqdm

In [51]:
posts_labels = []
count = 0
for i in tqdm(range(0, len(tsla_sentences), 100)):
  chunk = tsla_sentences[i:i + 100]
  count = count + 1

  tf_batch = tokenizer(chunk, max_length=128, padding=True, truncation=True, return_tensors='tf')
  tf_outputs = model(tf_batch)
  tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
  labels = [0,1]
  label = tf.argmax(tf_predictions, axis=1)

  pred_labels = label.numpy().tolist()

  posts_labels = posts_labels + pred_labels


print(len(posts_labels))

100%|██████████| 1319/1319 [18:46<00:00,  1.17it/s]

131870


In [52]:
print(posts_labels[:10])

[1, 1, 0, 1, 1, 1, 1, 1, 0, 1]


In [53]:
TSLA_tweets['label'] = posts_labels

In [54]:
TSLA_tweets.head(20)

,date,tweet,clean_post,label
0,2020-01-02,$SPY #SPY Daily vs. Monthly https://t.co/uOqC...,$SPY #SPY Daily vs. Monthly,1
1,2020-01-02,What a relaxing day off. Best part is the $SPY...,What a relaxing day off. Best part is the $SPY...,1
2,2020-01-02,$SPX managed to eek out a close above that LT ...,$SPX managed to eek out a close above that LT ...,0
3,2020-01-02,Mercury Bands &amp; Algorithmic Pivots ES Futu...,Mercury Bands &amp; Algorithmic Pivots ES Futu...,1
4,2020-01-02,2020's Key Global Events: A Calendar For The Y...,2020's Key Global Events: A Calendar For The Y...,1
5,2020-01-02,$SPY $ES_F Welcome 2020! https://t.co/bcLUFY...,$SPY $ES_F Welcome 2020!,1
6,2020-01-02,(AMGN) Upgraded to Buy: What you Need to Know ...,(AMGN) Upgraded to Buy: What you Need to Know ...,1
7,2020-01-02,Charts and coffee tonight! Probably around 10p...,Charts and coffee tonight! Probably around 10p...,1
8,2020-01-02,The P/E on the S&amp;P 500 $SPY is down relati...,The P/E on the S&amp;P 500 $SPY is down relati...,0
9,2020-01-02,$SPX 500 ( X 3 ) + $SPY - update ( primeras g...,$SPX 500 ( X 3 ) + $SPY - update ( primeras g...,1


In [55]:
TSLA_tweets.to_csv('/content/drive/MyDrive/TFG/BERT/tweets_posts/label_2020_SPY_posts.csv', index=False)